# 1. pytorch两大法宝
- dir(): 打开，看见
- help(): 说明书

In [1]:
import torch
torch.cuda.is_available()

True

In [5]:
dir(torch.cuda.is_available)

['__annotations__',
 '__builtins__',
 '__call__',
 '__class__',
 '__closure__',
 '__code__',
 '__defaults__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__get__',
 '__getattribute__',
 '__globals__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__kwdefaults__',
 '__le__',
 '__lt__',
 '__module__',
 '__name__',
 '__ne__',
 '__new__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

In [6]:
help(torch.cuda.is_available)

Help on function is_available in module torch.cuda:

is_available() -> bool
    Returns a bool indicating if CUDA is currently available.



# 2. 加载数据
## 2.1 Dataset
提供一种方式去获取数据及其label
- 如何获取每一个数据及其label
- 总共有多少个数据


In [14]:
from torch.utils.data import Dataset
from PIL import Image
import os
help(Dataset)

Help on class Dataset in module torch.utils.data.dataset:

class Dataset(typing.Generic)
 |  An abstract class representing a :class:`Dataset`.
 |  
 |  All datasets that represent a map from keys to data samples should subclass
 |  it. All subclasses should overwrite :meth:`__getitem__`, supporting fetching a
 |  data sample for a given key. Subclasses could also optionally overwrite
 |  :meth:`__len__`, which is expected to return the size of the dataset by many
 |  :class:`~torch.utils.data.Sampler` implementations and the default options
 |  of :class:`~torch.utils.data.DataLoader`.
 |  
 |  .. note::
 |    :class:`~torch.utils.data.DataLoader` by default constructs a index
 |    sampler that yields integral indices.  To make it work with a map-style
 |    dataset with non-integral indices/keys, a custom sampler must be provided.
 |  
 |  Method resolution order:
 |      Dataset
 |      typing.Generic
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __add__(self, oth

In [23]:
class MyData(Dataset):
    def __init__(self, root_dir, label_dir):
        self.root_dir = root_dir
        self.label_dir = label_dir
        self.path = os.path.join(root_dir, label_dir)  # 到目录级
        self.img_path = os.listdir(self.path)  # 获取目录中所有文件名称
        pass

    def __getitem__(self, idx):  # 返回img和label
        img_name = self.img_path[idx]
        img_item_path = os.path.join(self.root_dir, self.label_dir, img_name)
        img = Image.open(img_item_path)
        label = self.label_dir
        return img, label

    def __len__(self):  # 返回给定数据集的大小
        return len(self.img_path)


In [24]:
root_dir = r"hymenoptera_data/train"
ants_label_dir = "ants"
bees_label_dir = "bees"
ants_dataset = MyData(root_dir, ants_label_dir)  # Dataset实例化
bees_dataset = MyData(root_dir, bees_label_dir)  # Dataset实例化

train_dataset = ants_dataset + bees_dataset

In [28]:
img, label = ants_dataset[0]
img.show()

## 2.2 DataLoader
获取数据的时候进行打包，为网络提供不同的数据形式

常用参数包括：
- batch_size: 每次取几个数据
- shuffle: 取数据是否打乱
- num_workers: 多进程计算，默认为0
- drop_last: 多出来的数据要不要舍去

In [14]:
from torch.utils.data import DataLoader
import torchvision
from torch.utils.tensorboard import SummaryWriter

test_data = torchvision.datasets.CIFAR10('./dataset', train=False, transform=torchvision.transforms.ToTensor())
test_loader = DataLoader(dataset=test_data, batch_size=64, shuffle=True, num_workers=0, drop_last=False)

img, target = test_data[0]
i = 0
for data in test_loader:  # test_loader无法用[i]索引
    imgs, targets = data
    i = i + 1
    if i > 0.5:
        break
        pass
print(img.shape, target)
print(imgs.shape, targets)

writer = SummaryWriter('logs')

for epoch in range(2):
    step = 0
    for data in test_loader:
        imgs, targets = data
        writer.add_images('Epoch: {}'.format(epoch), imgs, step)
        step = step + 1

torch.Size([3, 32, 32]) 3
torch.Size([64, 3, 32, 32]) tensor([7, 1, 3, 6, 7, 9, 0, 6, 5, 1, 5, 1, 8, 6, 7, 3, 8, 6, 3, 3, 4, 1, 4, 8,
        6, 2, 0, 3, 8, 2, 4, 0, 0, 3, 0, 0, 0, 5, 1, 1, 1, 6, 7, 1, 1, 4, 4, 7,
        3, 0, 0, 1, 3, 5, 1, 7, 0, 1, 3, 7, 4, 5, 8, 2])


# 3. Tensor Board使用
## 3.1 写函数

In [33]:
from torch.utils.tensorboard import SummaryWriter
# help(SummaryWriter)
writer = SummaryWriter("logs")
# writer.add_image()
for i in range(100):
    writer.add_scalar('y=x', i, i)
writer.close()

## 3.2 写图像

In [37]:
import numpy as np
from torch.utils.tensorboard import SummaryWriter
# help(SummaryWriter)
writer = SummaryWriter("logs")
img_path = r"hymenoptera_data/train/ants/0013035.jpg"
img_PIL = Image.open(img_path)
img_array = np.array(img_PIL)
print(img_array.shape, type(img_array))

writer.add_image(tag='figure', img_tensor=img_array, dataformats="HWC")
# for i in range(100):
#     writer.add_scalar('y=x', i, i)
writer.close()

(512, 768, 3) <class 'numpy.ndarray'>


# 4. Transform 使用

一个工具箱，里面有很多工具，例如totensor, resize等。多用于处理图片。

通过transforms.ToTensor解决两个问题：
- transform怎么使用
- 为什么我们需要Tensor数据类型

学习方法:
- 关注输入和输出类型
- 多看官方文档
- 关注方法需要什么参数

## 4.1 ToTensor

In [ ]:
from PIL import Image
from torch.utils.tensorboard import SummaryWriter
from torchvision import transforms
import cv2
img_path = r"hymenoptera_data/train/ants/5650366_e22b7e1065.jpg"
img = Image.open(img_path)
cv_img = cv2.imread(img_path)
# img.show()
# print(img)

tensor_trans = transforms.ToTensor()
tensor_img = tensor_trans(img)
tensor_img_cv = tensor_trans(cv_img)
# tensor_img

writer = SummaryWriter("logs")
writer.add_image(tag='ToTensor', img_tensor=tensor_img, dataformats="CHW",global_step=0)  # 两个图片颜色不一样，因为PIL和cv2用的颜色格式不同
writer.add_image(tag='ToTensor', img_tensor=tensor_img_cv, dataformats="CHW",global_step=1)
writer.close()

## 4.2 Normalize

In [ ]:
img_path = r"hymenoptera_data/train/ants/5650366_e22b7e1065.jpg"
img = Image.open(img_path)

tensor_trans = transforms.ToTensor()
tensor_img = tensor_trans(img)
trans_norm = transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
norm_tensor_img = trans_norm(tensor_img)
# tensor_img

writer = SummaryWriter("logs")
writer.add_image(tag='normal', img_tensor=tensor_img, dataformats="CHW",global_step=0)  # 两个图片颜色不一样，因为PIL和cv2用的颜色格式不同
writer.add_image(tag='normal', img_tensor=norm_tensor_img, dataformats="CHW",global_step=1)
writer.close()

## 4.3 Resize

In [ ]:
img_path = r"hymenoptera_data/train/ants/5650366_e22b7e1065.jpg"
img = Image.open(img_path)

tensor_trans = transforms.ToTensor()
tensor_img = tensor_trans(img)
# trans_norm = transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
# norm_tensor_img = trans_norm(tensor_img)
trans_resize = transforms.Resize([512, 512])
resize_tensor_img = trans_resize(tensor_img)
# tensor_img

writer = SummaryWriter("logs")
writer.add_image(tag='resize', img_tensor=tensor_img, dataformats="CHW",global_step=0)  # 两个图片颜色不一样，因为PIL和cv2用的颜色格式不同
writer.add_image(tag='resize', img_tensor=resize_tensor_img, dataformats="CHW",global_step=1)
writer.close()

## 4.4 Compose

Compose就是把多个Transforms组合在一起使用。

In [ ]:
from PIL import Image
from torch.utils.tensorboard import SummaryWriter
from torchvision import transforms
import cv2
img_path = r"hymenoptera_data/train/ants/5650366_e22b7e1065.jpg"
img = Image.open(img_path)
# img.show()
# print(img)

tensor_trans = transforms.ToTensor()
normal_trans = transforms.Normalize([0.5,0.5,0.5], [0.5,0.5,0.5])
resize_trans = transforms.Resize([512, 512])
compose_trans = transforms.Compose([tensor_trans, normal_trans, resize_trans])

trans_img = compose_trans(img)
# tensor_img

writer = SummaryWriter("logs")
writer.add_image(tag='Compose', img_tensor=tensor_img, dataformats="CHW",global_step=0)  # 两个图片颜色不一样，因为PIL和cv2用的颜色格式不同
writer.add_image(tag='Compose', img_tensor=trans_img, dataformats="CHW",global_step=1)
writer.close()

# 5. 数据集的使用

这里用torchvision为例进行说明

In [3]:
import torchvision

dataset_transform = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor()
])

train_set = torchvision.datasets.CIFAR10(root='./dataset', train=True, download=True, transform=dataset_transform)
test_set = torchvision.datasets.CIFAR10(root='./dataset', train=False, download=True, transform=dataset_transform)

print(test_set[0])

Files already downloaded and verified
Files already downloaded and verified
(tensor([[[0.6196, 0.6235, 0.6471,  ..., 0.5373, 0.4941, 0.4549],
         [0.5961, 0.5922, 0.6235,  ..., 0.5333, 0.4902, 0.4667],
         [0.5922, 0.5922, 0.6196,  ..., 0.5451, 0.5098, 0.4706],
         ...,
         [0.2667, 0.1647, 0.1216,  ..., 0.1490, 0.0510, 0.1569],
         [0.2392, 0.1922, 0.1373,  ..., 0.1020, 0.1137, 0.0784],
         [0.2118, 0.2196, 0.1765,  ..., 0.0941, 0.1333, 0.0824]],

        [[0.4392, 0.4353, 0.4549,  ..., 0.3725, 0.3569, 0.3333],
         [0.4392, 0.4314, 0.4471,  ..., 0.3725, 0.3569, 0.3451],
         [0.4314, 0.4275, 0.4353,  ..., 0.3843, 0.3725, 0.3490],
         ...,
         [0.4863, 0.3922, 0.3451,  ..., 0.3804, 0.2510, 0.3333],
         [0.4549, 0.4000, 0.3333,  ..., 0.3216, 0.3216, 0.2510],
         [0.4196, 0.4118, 0.3490,  ..., 0.3020, 0.3294, 0.2627]],

        [[0.1922, 0.1843, 0.2000,  ..., 0.1412, 0.1412, 0.1294],
         [0.2000, 0.1569, 0.1765,  ..., 0.1216

# 6. 网络搭建

## 6.1 小试牛刀

In [17]:
from torch import nn
import torch

class Tudui(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, input):
        output = input + 1
        return output

tudui = Tudui()
x = torch.tensor(1.0)
print(tudui(x))

tensor(2.)


## 6.2 卷积层

In [18]:
import torch
import torchvision


dataset = torchvision.datasets.CIFAR10('./dataset', train=False, transform=torchvision.transforms.ToTensor(), download=False)
dataloader = DataLoader(dataset, batch_size=64)

class Tudui(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=6, kernel_size=3, stride=1, padding=0)
        pass

    def forward(self, x):
        x = self.conv1(x)
        return x

tudui = Tudui()
print(tudui)

for data in dataloader:
    imgs, targets = data
    output = tudui(imgs)
    print(output.shape)
    pass

Tudui(
  (conv1): Conv2d(3, 6, kernel_size=(3, 3), stride=(1, 1))
)


## 6.3 池化层

In [19]:
import torch
import torchvision
from torch import nn
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

dataset = torchvision.datasets.CIFAR10("dataset", train=False, download=False, transform=torchvision.transforms.ToTensor())

dataloader = DataLoader(dataset, batch_size=64, shuffle=True)



# input = torch.tensor([[1, 2, 0, 3, 1],
#                       [0, 1, 2, 3, 1],
#                       [1, 2, 1, 0, 0],
#                       [5, 2, 3, 1, 1],
#                       [2, 1, 0, 1, 1]], dtype=torch.float32)
#
# input = torch.reshape(input, (-1, 1, 5, 5))
# print(input.shape)

class Wu(nn.Module):

    def __init__(self):
        super(Wu, self).__init__()
        self.maxpool1 = nn.MaxPool2d(kernel_size=3, ceil_mode=True)

    def forward(self, input):
        output = self.maxpool1(input)
        return output


wu = Wu()
# output = wu(input)
# print(output)

writer = SummaryWriter("logs")

step = 0
for data in dataloader:
    imgs, targets = data
    writer.add_images("input", imgs, step)
    output = wu(imgs)
    writer.add_images("output", output, step)
    step = step + 1
writer.close()

## 6.4 非线性激活

ReLU, Sigmoid ...

## 6.5 线性层

In [21]:
import torch
import torchvision.datasets
from torch import nn
from torch.nn import Linear
from torch.utils.data import DataLoader

dataset = torchvision.datasets.CIFAR10("dataset", train=False, transform=torchvision.transforms.ToTensor(), download=False)

dataloader = DataLoader(dataset, batch_size=64, drop_last=True)

class Wu(nn.Module):
    def __init__(self):
        super(Wu, self).__init__()
        self.linear1 = Linear(196608, 10)

    def forward(self, input):
        output = self.linear1(input)
        return output

wu = Wu()


for data in dataloader:
    imgs, targets = data
    print(imgs.shape)
    # output = torch.reshape(imgs, (1, 1, 1, -1))  # 展平
    output = torch.flatten(imgs)  # 展平，和上面的reshape本质上是一样的，但更快更推荐
    print(output.shape)
    output = wu(output)
    print(output.shape)

    # 这里报错是因为最后一个batch只有16个（其他都有64个）

torch.Size([64, 3, 32, 32])
torch.Size([196608])
torch.Size([10])
torch.Size([64, 3, 32, 32])
torch.Size([196608])
torch.Size([10])
torch.Size([64, 3, 32, 32])
torch.Size([196608])
torch.Size([10])
torch.Size([64, 3, 32, 32])
torch.Size([196608])
torch.Size([10])
torch.Size([64, 3, 32, 32])
torch.Size([196608])
torch.Size([10])
torch.Size([64, 3, 32, 32])
torch.Size([196608])
torch.Size([10])
torch.Size([64, 3, 32, 32])
torch.Size([196608])
torch.Size([10])
torch.Size([64, 3, 32, 32])
torch.Size([196608])
torch.Size([10])
torch.Size([64, 3, 32, 32])
torch.Size([196608])
torch.Size([10])
torch.Size([64, 3, 32, 32])
torch.Size([196608])
torch.Size([10])
torch.Size([64, 3, 32, 32])
torch.Size([196608])
torch.Size([10])
torch.Size([64, 3, 32, 32])
torch.Size([196608])
torch.Size([10])
torch.Size([64, 3, 32, 32])
torch.Size([196608])
torch.Size([10])
torch.Size([64, 3, 32, 32])
torch.Size([196608])
torch.Size([10])
torch.Size([64, 3, 32, 32])
torch.Size([196608])
torch.Size([10])
torch.Size

## 6.6 nn搭建实践 —— 以CIFAR10为例

In [22]:
import torch
from torch import nn
from torch.nn import Conv2d, MaxPool2d, Flatten, Linear, Sequential
from torch.utils.tensorboard import SummaryWriter


class Wu(nn.Module):
    def __init__(self):
        super(Wu, self).__init__()
        self.conv1 = Conv2d(3, 32, 5, 1, 2)
        self.maxpool1 = MaxPool2d(2, ceil_mode=True)  # celi_model用于设置余数要不要算进来计算，True是要的意思
        self.conv2 = Conv2d(32, 32, 5, 1, 2)
        self.maxpool2 = MaxPool2d(2, ceil_mode=True)
        self.conv3 = Conv2d(32, 64, 5, 1, 2)
        self.maxpool3 = MaxPool2d(2, ceil_mode=True)
        self.flatten = Flatten()
        self.linear1 = Linear(1024, 64)
        self.linear2 = Linear(64, 10)

        self.model1 = Sequential(
            Conv2d(3, 32, 5, 1, 2),
            MaxPool2d(2, ceil_mode=True),
            Conv2d(32, 32, 5, 1, 2),
            MaxPool2d(2, ceil_mode=True),
            Conv2d(32, 64, 5, 1, 2),
            MaxPool2d(2, ceil_mode=True),
            Flatten(),
            Linear(1024, 64),
            Linear(64, 10)
        )

    def forward(self, x):
        # x = self.conv1(x)
        # x = self.maxpool1(x)
        # x = self.conv2(x)
        # x = self.maxpool2(x)
        # x = self.conv3(x)
        # x = self.maxpool3(x)
        # x = self.flatten(x)
        # x = self.linear1(x)
        # x = self.linear2(x)

        x = self.model1(x)

        return x


wu = Wu()
print(wu)

# 测试网络的正确性
input = torch.ones((64, 3, 32, 32))
# output = wu(input)
# print(output.shape)

writer = SummaryWriter("logs")
writer.add_graph(wu, input)
writer.close()

Wu(
  (conv1): Conv2d(3, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (maxpool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=True)
  (conv2): Conv2d(32, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (maxpool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=True)
  (conv3): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (maxpool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=True)
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear1): Linear(in_features=1024, out_features=64, bias=True)
  (linear2): Linear(in_features=64, out_features=10, bias=True)
  (model1): Sequential(
    (0): Conv2d(3, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=True)
    (2): Conv2d(32, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, cei

# 7. 损失函数、反向传播、优化

## 7.1 Loss函数 和 backward()

- 计算实际输出和目标之间的差距
- 为我们更新输出提供一定的依据（反向传播）

In [2]:
import torch
import torchvision
from torch import nn
from torch.nn import Conv2d, MaxPool2d, Flatten, Linear, Sequential
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

dataset = torchvision.datasets.CIFAR10("dataset", train=False, transform=torchvision.transforms.ToTensor(), download=False)

dataloader = DataLoader(dataset, 64, True, drop_last=False)

class Wu(nn.Module):

    def __init__(self):
        super(Wu, self).__init__()
        # self.conv1 = Conv2d(3, 32, 5, 1, 2)
        # self.maxpool1 = MaxPool2d(2, ceil_mode=True)  # celi_model用于设置余数要不要算进来计算，True是要的意思
        # self.conv2 = Conv2d(32, 32, 5, 1, 2)
        # self.maxpool2 = MaxPool2d(2, ceil_mode=True)
        # self.conv3 = Conv2d(32, 64, 5, 1, 2)
        # self.maxpool3 = MaxPool2d(2, ceil_mode=True)
        # self.flatten = Flatten()
        # self.linear1 = Linear(1024, 64)
        # self.linear2 = Linear(64, 10)

        self.model1 = Sequential(
            Conv2d(3, 32, 5, 1, 2),
            MaxPool2d(2, ceil_mode=True),
            Conv2d(32, 32, 5, 1, 2),
            MaxPool2d(2, ceil_mode=True),
            Conv2d(32, 64, 5, 1, 2),
            MaxPool2d(2, ceil_mode=True),
            Flatten(),
            Linear(1024, 64),
            Linear(64, 10)
        )

    def forward(self, x):
        # x = self.conv1(x)
        # x = self.maxpool1(x)
        # x = self.conv2(x)
        # x = self.maxpool2(x)
        # x = self.conv3(x)
        # x = self.maxpool3(x)
        # x = self.flatten(x)
        # x = self.linear1(x)
        # x = self.linear2(x)

        x = self.model1(x)

        return x


wu = Wu()
loss = nn.CrossEntropyLoss()

for data in dataloader:
    imgs, targets = data
    # print(imgs.shape, targets.shape)
    output = wu(imgs)
    # print(output)
    result_loss = loss(output, targets)
    # print(output.shape)
    print(result_loss)
    result_loss.backward()  # 反向传播求梯度
    print()

tensor(2.2848, grad_fn=<NllLossBackward0>)

tensor(2.3066, grad_fn=<NllLossBackward0>)

tensor(2.3045, grad_fn=<NllLossBackward0>)

tensor(2.2874, grad_fn=<NllLossBackward0>)

tensor(2.3014, grad_fn=<NllLossBackward0>)

tensor(2.3073, grad_fn=<NllLossBackward0>)

tensor(2.3103, grad_fn=<NllLossBackward0>)

tensor(2.3099, grad_fn=<NllLossBackward0>)

tensor(2.3069, grad_fn=<NllLossBackward0>)

tensor(2.3064, grad_fn=<NllLossBackward0>)

tensor(2.3154, grad_fn=<NllLossBackward0>)

tensor(2.3071, grad_fn=<NllLossBackward0>)

tensor(2.2923, grad_fn=<NllLossBackward0>)

tensor(2.2969, grad_fn=<NllLossBackward0>)

tensor(2.2920, grad_fn=<NllLossBackward0>)

tensor(2.3023, grad_fn=<NllLossBackward0>)

tensor(2.3086, grad_fn=<NllLossBackward0>)

tensor(2.3011, grad_fn=<NllLossBackward0>)

tensor(2.3033, grad_fn=<NllLossBackward0>)

tensor(2.3043, grad_fn=<NllLossBackward0>)

tensor(2.3003, grad_fn=<NllLossBackward0>)

tensor(2.3070, grad_fn=<NllLossBackward0>)

tensor(2.3066, grad_fn=<NllLossB

## 7.2 优化器

这一节包括很多超参数的设置。

In [3]:
import torch
import torchvision
from torch import nn
from torch.nn import Conv2d, MaxPool2d, Flatten, Linear, Sequential
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

dataset = torchvision.datasets.CIFAR10("dataset", train=False, transform=torchvision.transforms.ToTensor(), download=False)

dataloader = DataLoader(dataset, 64, True, drop_last=False)

class Wu(nn.Module):

    def __init__(self):
        super(Wu, self).__init__()
        # self.conv1 = Conv2d(3, 32, 5, 1, 2)
        # self.maxpool1 = MaxPool2d(2, ceil_mode=True)  # ceil_model用于设置余数要不要算进来计算，True是要的意思
        # self.conv2 = Conv2d(32, 32, 5, 1, 2)
        # self.maxpool2 = MaxPool2d(2, ceil_mode=True)
        # self.conv3 = Conv2d(32, 64, 5, 1, 2)
        # self.maxpool3 = MaxPool2d(2, ceil_mode=True)
        # self.flatten = Flatten()
        # self.linear1 = Linear(1024, 64)
        # self.linear2 = Linear(64, 10)

        self.model1 = Sequential(
            Conv2d(3, 32, 5, 1, 2),
            MaxPool2d(2, ceil_mode=True),
            Conv2d(32, 32, 5, 1, 2),
            MaxPool2d(2, ceil_mode=True),
            Conv2d(32, 64, 5, 1, 2),
            MaxPool2d(2, ceil_mode=True),
            Flatten(),
            Linear(1024, 64),
            Linear(64, 10)
        )

    def forward(self, x):
        # x = self.conv1(x)
        # x = self.maxpool1(x)
        # x = self.conv2(x)
        # x = self.maxpool2(x)
        # x = self.conv3(x)
        # x = self.maxpool3(x)
        # x = self.flatten(x)
        # x = self.linear1(x)
        # x = self.linear2(x)

        x = self.model1(x)

        return x


wu = Wu()
loss = nn.CrossEntropyLoss()
optim = torch.optim.SGD(wu.parameters(), lr=0.01)

for epoch in range(20):
    running_loss = 0.0  # 看每一轮的loss是多少
    for data in dataloader:
        imgs, targets = data
        output = wu(imgs)
        result_loss = loss(output, targets)
        optim.zero_grad()
        result_loss.backward()
        optim.step()
        # print(result_loss)
        running_loss = running_loss + result_loss
    print(running_loss)

tensor(361.0699, grad_fn=<AddBackward0>)
tensor(357.8331, grad_fn=<AddBackward0>)
tensor(342.2478, grad_fn=<AddBackward0>)
tensor(320.7163, grad_fn=<AddBackward0>)
tensor(313.1638, grad_fn=<AddBackward0>)
tensor(304.2767, grad_fn=<AddBackward0>)
tensor(296.5301, grad_fn=<AddBackward0>)
tensor(288.4012, grad_fn=<AddBackward0>)
tensor(280.5760, grad_fn=<AddBackward0>)
tensor(273.1507, grad_fn=<AddBackward0>)
tensor(266.6805, grad_fn=<AddBackward0>)
tensor(262.1871, grad_fn=<AddBackward0>)
tensor(257.3079, grad_fn=<AddBackward0>)
tensor(252.0457, grad_fn=<AddBackward0>)
tensor(248.0773, grad_fn=<AddBackward0>)
tensor(244.2853, grad_fn=<AddBackward0>)
tensor(240.3637, grad_fn=<AddBackward0>)
tensor(237.6151, grad_fn=<AddBackward0>)
tensor(234.4866, grad_fn=<AddBackward0>)
tensor(229.6207, grad_fn=<AddBackward0>)


# 8. 现有模型的使用与修改——迁移学习

In [2]:
import torch
import torchvision
from torch import nn
from torch.nn import Conv2d, MaxPool2d, Flatten, Linear, Sequential
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

vgg16_false = torchvision.models.vgg16(pretrained=False)
vgg16_true = torchvision.models.vgg16(pretrained=True)

print(vgg16_false)

# 添加Modeule：
vgg16_true.add_module('add_linear', nn.Linear(1000,10))

# 修改Module：
vgg16_false.classifier[6] = nn.Linear(in_features=4096, out_features=10, bias=True)

print(vgg16_true)
print(vgg16_false)

c:\Users\62411\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\62411\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
c:\Users\62411\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

# 9. 网络模型的保存与加载

## 9.1 模型的保存

In [3]:
import torch
import torchvision

vgg16 = torchvision.models.vgg16(pretrained=True)

# 方法一：
torch.save(vgg16, "vgg16_method1.pth")

# 方法二：
torch.save(vgg16.state_dict(), "vgg16_method2.pth")

c:\Users\62411\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\62411\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


## 9.2 模型的加载

In [5]:
# 方法一：
model1 = torch.load("vgg16_method1.pth")
print(model1)

# 放法二：
model2 = torchvision.models.vgg16(pretrained=False)  # 要先重新搭一遍模型的框架
model2.load_state_dict(torch.load("vgg16_method2.pth"))
print(model2)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

c:\Users\62411\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\62411\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

# 10. 模型整体

In [8]:
# 第一种方法：gpu训练三处加cuda() 分别是：网络模型、损失函数、数据
# 第二种方法：device = 'cuda' 也是三处：网络模型、损失函数、数据

import torch.optim.optimizer
from torch.utils.tensorboard import SummaryWriter
import torch
import torchvision.datasets
from torch import nn
from torch.utils.data import DataLoader
import time


# device = torch.device('cpu')
# device = torch.device('cuda')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# from model import *

train_data = torchvision.datasets.CIFAR10("dataset", train=True, transform=torchvision.transforms.ToTensor(),
                                          download=True)
test_data = torchvision.datasets.CIFAR10("dataset", train=False, transform=torchvision.transforms.ToTensor(),
                                         download=True)

train_data_size = len(train_data)  # 直接获取数据集的大小
test_data_size = len(test_data)
print("训练数据集的长度为：{}".format(train_data_size))
print("测试数据集的长度为：{}".format(test_data_size))

train_dataloader = DataLoader(train_data, 64, shuffle=True)
test_dataloader = DataLoader(test_data, 64, shuffle=True)


class Wu(nn.Module):
    def __init__(self):
        super(Wu, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(3, 32, 5, 1, 2),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 32, 5, 1, 2),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 5, 1, 2),
            nn.MaxPool2d(2),
            nn.Flatten(),
            nn.Linear(64 * 4 * 4, 64),
            nn.Linear(64, 10)
        )

    def forward(self, x):
        x = self.model(x)
        return x

    pass


wu = Wu()
if torch.cuda.is_available():
    # wu = wu.cuda()  # gpu训练1
    wu = wu.to(device)

loss_fn = nn.CrossEntropyLoss()
if torch.cuda.is_available():
    # loss_fn = loss_fn.cuda()  # gpu训练2
    loss_fn = loss_fn.to(device)

optimizer = torch.optim.SGD(wu.parameters(), lr=0.01)

total_train_step = 0
total_test_step = 0
epoch = 20

writer = SummaryWriter("logs")

for i in range(epoch):
    print("第{}轮训练开始".format(i + 1))
    start_time = time.time()
    wu.train()
    for data in train_dataloader:
        imgs, targets = data
        if torch.cuda.is_available():
            # imgs = imgs.cuda()  # gpu训练3
            # targets = targets.cuda()  # gpu训练3
            imgs = imgs.to(device)
            targets = targets.to(device)
        outputs = wu(imgs)
        loss = loss_fn(outputs, targets)

        # 优化器优化模型
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (total_train_step + 1) % 100 == 0:
            print("训练次数:{}, Loss:{}".format(total_train_step + 1, loss.item()))
            writer.add_scalar("train_loss", loss.item(), total_train_step)
        total_train_step = total_train_step + 1

    wu.eval()
    total_test_loss = 0
    total_accuracy = 0
    with torch.no_grad():
        for data in test_dataloader:
            imgs, targets = data
            if torch.cuda.is_available():
                # imgs = imgs.cuda()  # gpu训练3
                # targets = targets.cuda()  # gpu训练3
                imgs = imgs.to(device)
                targets = targets.to(device)
            outputs = wu(imgs)
            loss = loss_fn(outputs, targets)
            total_test_loss = total_test_loss + loss
            total_accuracy = total_accuracy + (outputs.argmax(1) == targets).sum()
    end_time = time.time()
    print("整体数据集上的Loss：{}".format(loss.item()))
    print("整体测试集上的正确率: {}".format(total_accuracy / test_data_size))
    print("本轮训练时长: {}".format(end_time-start_time))
    writer.add_scalar("test_loss", loss.item(), total_test_step)
    writer.add_scalar("test_accuracy", total_accuracy / test_data_size, total_test_step)
    total_test_step = total_test_step + 1

    # torch.save(wu, "wu_{}.pth".format(i))  # 保存每一次训练模型
    # torch.save(wu.state_dict(), "wu_{}".format(i))  # 另一种保存方式
    # print("模型已保存！")

torch.save(wu.state_dict(), "wu.pth")

writer.close()

Files already downloaded and verified
Files already downloaded and verified
训练数据集的长度为：50000
测试数据集的长度为：10000
第1轮训练开始
训练次数:100, Loss:2.28983736038208
训练次数:200, Loss:2.292184829711914
训练次数:300, Loss:2.2539641857147217
训练次数:400, Loss:2.2014808654785156
训练次数:500, Loss:2.149015188217163
训练次数:600, Loss:2.056422710418701
训练次数:700, Loss:1.921677827835083
整体数据集上的Loss：2.1426024436950684
整体测试集上的正确率: 0.29100000858306885
本轮训练时长: 55.716891050338745
第2轮训练开始
训练次数:800, Loss:1.8396490812301636
训练次数:900, Loss:1.9348477125167847
训练次数:1000, Loss:1.8176124095916748
训练次数:1100, Loss:1.8490245342254639
训练次数:1200, Loss:1.751731514930725
训练次数:1300, Loss:1.6793042421340942
训练次数:1400, Loss:1.8811403512954712
训练次数:1500, Loss:1.841321587562561
整体数据集上的Loss：1.628587245941162
整体测试集上的正确率: 0.38679999113082886
本轮训练时长: 56.8618278503418
第3轮训练开始
训练次数:1600, Loss:1.7468445301055908
训练次数:1700, Loss:1.7796196937561035
训练次数:1800, Loss:1.9693225622177124
训练次数:1900, Loss:1.696186900138855
训练次数:2000, Loss:1.7908596992492676
训练次数:2100

# 11. 模型验证

In [12]:
from PIL import Image
import torchvision
import torch

image_path = 'dog2.jpg'
image = Image.open(image_path)
print(image)

transforms = torchvision.transforms.Compose([
    torchvision.transforms.Resize((32,32)),
    torchvision.transforms.ToTensor()
])

image = transforms(image)
print(image.shape)

class Wu(nn.Module):
    def __init__(self):
        super(Wu, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(3, 32, 5, 1, 2),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 32, 5, 1, 2),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 5, 1, 2),
            nn.MaxPool2d(2),
            nn.Flatten(),
            nn.Linear(64 * 4 * 4, 64),
            nn.Linear(64, 10)
        )

    def forward(self, x):
        x = self.model(x)
        return x

    pass

model = Wu()
model.load_state_dict(torch.load("wu.pth"))
print(model)

image = torch.reshape(image, (1, 3, 32, 32))
model.eval()
with torch.no_grad():
    output = model(image)
print(output)
print(output.argmax(1))


<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=276x182 at 0x192C542A560>
torch.Size([3, 32, 32])
Wu(
  (model): Sequential(
    (0): Conv2d(3, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (2): Conv2d(32, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Flatten(start_dim=1, end_dim=-1)
    (7): Linear(in_features=1024, out_features=64, bias=True)
    (8): Linear(in_features=64, out_features=10, bias=True)
  )
)
tensor([[-7.3367, -2.9395, -0.0099,  4.1078,  5.0973,  6.2311, -0.7234,  2.6055,
         -7.2523, -0.3435]])
tensor([5])
